In [20]:
import numpy as np
import pandas as pd

In [21]:
df_august = pd.read_csv("https://raw.githubusercontent.com/CathieLC/Happy_four_friends/main/Dataset/ManoMano%20-%20August%202021.csv?token=GHSAT0AAAAAABMXSZJMU35CES7HVZWAJODMYPRFTPA")
df_september = pd.read_csv("https://raw.githubusercontent.com/CathieLC/Happy_four_friends/main/Dataset/ManoMano%20-%20September%202021.csv?token=GHSAT0AAAAAABMXSZJMUP5FCAR47B6L2AEQYPRFU5Q")
df_october = pd.read_csv("https://raw.githubusercontent.com/CathieLC/Happy_four_friends/main/Dataset/ManoMano%20-%20October%202021.csv?token=GHSAT0AAAAAABMXSZJMRDZONS4ZZPZ4U72UYPRFVFQ")
df_november = pd.read_csv("https://raw.githubusercontent.com/CathieLC/Happy_four_friends/main/Dataset/ManoMano%20-%20November%202021.csv?token=GHSAT0AAAAAABMXSZJM57XIDQDMXJ3CA5IKYPRFVMA")


C:\Users\Berenger\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [22]:
# Concatenation of all dataset
frames = [df_august, df_september, df_october, df_november]
df_total = pd.concat(frames)
# Display all columns when exploring datasets
pd.set_option('display.max_columns', None)

In [23]:
print (df_august.shape)
print (df_september.shape)
print (df_october.shape)
print (df_november.shape)

(17652, 35)
(19867, 34)
(24663, 34)
(33000, 34)


In [24]:
df_total.head()

,id,comment,original_comment,score,data_scale,data_source,created_at,date,day,month,is_mf,device,family,is_b2b,reason,browser,zipcode,category,language,platform,provider,first_order,nb_articles,csat_presales,shipping_fees,bv_transaction,payment_method,operating_system,last_paid_channel,has_presales_contact,shipping_fees_bucket,bv_transaction_bucket,has_manodvisor_contact,themes,tags
0,295564331,NaN,NaN,10,NPS,Customer Effort Score,2021-08-01T00:02:13.000Z,2021-08-01,dimanche,août,False,phone,Mobilier d'intérieur,False,NaN,Google 170.0.386351093,BT80 9DH,Meuble de salle à manger,Unknown,GB,Deuba (GB),True,1,FALSE,0,"119,95",paypal,iOS 14.6,SHOPBOT,False,0,101 - 150,False,NaN,NaN
1,295563489,NaN,NaN,10,NPS,Customer Effort Score,2021-08-01T00:04:10.000Z,2021-08-01,dimanche,août,False,desktop,Mobilier d'intérieur,False,NaN,Chrome 92.0.4515,SO31 9LB,Meuble de bureau,Unknown,GB,Songmics (GB),True,1,FALSE,0,"99,99",mastercard,Windows 10,SHOPBOT,False,0,1 - 100,False,NaN,NaN
2,295564358,NaN,NaN,8,NPS,Customer Effort Score,2021-08-01T00:15:48.000Z,2021-08-01,dimanche,août,False,desktop,Mobilier d'intérieur,False,NaN,Chrome 92.0.4515,64160,Meuble de salle à manger,Unknown,FR,vidaXL (FR),True,1,FALSE,0,"84,99",visa,Windows 10,AFFILIATION,False,0,1 - 100,False,NaN,NaN
3,295563988,NaN,NaN,5,NPS,Customer Effort Score,2021-08-01T02:04:17.000Z,2021-08-01,dimanche,août,False,phone,Jardin piscine,False,I had trouble signing up or logging in,Mobile Safari 14.1,HU13 9LT,Mobilier de jardin et jeux,Unknown,GB,Agito UK (GB),True,1,FALSE,0,"92,99",visa,iOS 14.5.1,SHOPBOT,False,0,1 - 100,False,NaN,Detractor '- Signup/Login
4,295562040,NaN,NaN,10,NPS,Customer Effort Score,2021-08-01T02:38:07.000Z,2021-08-01,dimanche,août,False,desktop,Plomberie chauffage,False,NaN,Chrome 92.0.4515,14480,Chauffage et climatisation,Unknown,FR,Ecoland's (FR),False,1,FALSE,"1,92","8,3",paypal,Windows 10,SEM,False,1 - 5,1 - 100,False,NaN,NaN


In [25]:
df_total.columns

Index(['id', 'comment', 'original_comment', 'score', 'data_scale',
       'data_source', 'created_at', 'date', 'day', 'month', 'is_mf', 'device',
       'family', 'is_b2b', 'reason', 'browser', 'zipcode', 'category',
       'language', 'platform', 'provider', 'first_order', 'nb_articles',
       'csat_presales', 'shipping_fees', 'bv_transaction', 'payment_method',
       'operating_system', 'last_paid_channel', 'has_presales_contact',
       'shipping_fees_bucket', 'bv_transaction_bucket',
       'has_manodvisor_contact', 'themes', 'tags'],
      dtype='object')

In [26]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95182 entries, 0 to 32999
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      95182 non-null  int64 
 1   comment                 11507 non-null  object
 2   original_comment        11796 non-null  object
 3   score                   95182 non-null  int64 
 4   data_scale              95182 non-null  object
 5   data_source             95182 non-null  object
 6   created_at              95182 non-null  object
 7   date                    95182 non-null  object
 8   day                     95182 non-null  object
 9   month                   95182 non-null  object
 10  is_mf                   95182 non-null  bool  
 11  device                  95182 non-null  object
 12  family                  95176 non-null  object
 13  is_b2b                  94508 non-null  object
 14  reason                  3976 non-null   object
 15  br

In [27]:
df_total.describe()

,id,score,nb_articles
count,9.518200e+04,95182.000000,95182.000000
mean,3.355182e+08,8.969721,1.322183
std,2.117025e+07,1.545100,1.027572
min,2.955617e+08,0.000000,1.000000
25%,3.201404e+08,8.000000,1.000000
50%,3.370718e+08,10.000000,1.000000
75%,3.552651e+08,10.000000,1.000000
max,3.690581e+08,10.000000,39.000000


In [28]:
df_total.isna().sum()

id                            0
comment                   83675
original_comment          83386
score                         0
data_scale                    0
data_source                   0
created_at                    0
date                          0
day                           0
month                         0
is_mf                         0
device                        0
family                        6
is_b2b                      674
reason                    91206
browser                       0
zipcode                   77530
category                      6
language                      0
platform                      0
provider                      6
first_order                   0
nb_articles                   0
csat_presales                 0
shipping_fees                 0
bv_transaction                0
payment_method                0
operating_system              0
last_paid_channel           722
has_presales_contact          0
shipping_fees_bucket          0
bv_trans

In [31]:
df_dropped_na = df_total.dropna(subset=["comment"])
df_dropped_na

,id,comment,original_comment,score,data_scale,data_source,created_at,date,day,month,is_mf,device,family,is_b2b,reason,browser,zipcode,category,language,platform,provider,first_order,nb_articles,csat_presales,shipping_fees,bv_transaction,payment_method,operating_system,last_paid_channel,has_presales_contact,shipping_fees_bucket,bv_transaction_bucket,has_manodvisor_contact,themes,tags
15,295562096,Clear and accurate... thank you.,Clair et précis..... merci,10,NPS,Customer Effort Score,2021-08-01T06:05:11.000Z,2021-08-01,dimanche,août,False,phone,Jardin piscine,False,NaN,Chrome Mobile 92.0.4515,7700,Barbecue et plancha,French,FR,vidaXL (FR),False,2,FALSE,0,"102,49",visa,Android 8.0.0,SEM,False,0,101 - 150,False,Delivery > Order Accuracy > 1;Online Experienc...,NaN
23,295562415,All right.,Très bien,9,NPS,Customer Effort Score,2021-08-01T06:33:25.000Z,2021-08-01,dimanche,août,False,desktop,Jardin piscine,False,NaN,Edge 92.0.902,55300,Matériel d'arrosage et récupérateurs d'eau,French,FR,Multitanks (FR),False,1,FALSE,0,"44,55",paypal,Windows 10,SHOPBOT,False,0,1 - 100,False,Service > General Satisfaction > 1,NaN
24,295563199,All right.,trés bien,9,NPS,Customer Effort Score,2021-08-01T06:34:44.000Z,2021-08-01,dimanche,août,False,desktop,Construction matériaux,False,NaN,Edge 92.0.902,55300,"Toiture, gouttière et bardage",French,FR,Briday FR (FR),False,1,FALSE,0,"24,89",paypal,Windows 10,SHOPBOT,False,0,1 - 100,False,Service > General Satisfaction > 1,NaN
36,295562240,First class service. Have a gold medal.,First class service. Have a gold medal.,10,NPS,Customer Effort Score,2021-08-01T07:09:21.000Z,2021-08-01,dimanche,août,False,desktop,Plomberie chauffage,False,NaN,Firefox 89.0,SN16 9JZ,Chauffage et climatisation,English,GB,DIRECT2PUBLIK LTD (GB),False,1,FALSE,0,"17,95",paypal,Mac OS X 10.13,EMAIL,False,0,1 - 100,False,Service > General Satisfaction > 1,NaN
39,295562144,Perfect except 2 sends different for an order,Parfait sauf 2 envoie différent pour une commande,9,NPS,Customer Effort Score,2021-08-01T07:19:17.000Z,2021-08-01,dimanche,août,True,tablet,Plomberie chauffage,False,NaN,Chrome 92.0.4515,26100,Robinetterie,French,FR,Silamp (FR),False,2,FALSE,0,"101,78",visa,Android 11,SEM,False,0,101 - 150,False,Service > General Satisfaction > 1;Delivery > ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32940,369056375,Very simple and practical,Molto semplice e pratico,10,NPS,Customer Effort Score,2021-11-30T21:48:12.000Z,2021-11-30,mardi,novembre,False,phone,Jardin piscine,False,NaN,Chrome Mobile WebView 96.0.4664,NaN,Mobilier de jardin et jeux,Italian,IT,Produce Shop (IT),False,1,neutral,0,"69,95",mastercard,Android 11,AFFILIATION,False,0,1 - 100,False,Service > Convenience > 1;Online Experience > ...,NaN
32966,369050246,"All right, both the attention and the simplici...","Todo correcto, tanto la atención como la senci...",10,NPS,Customer Effort Score,2021-11-30T22:22:47.000Z,2021-11-30,mardi,novembre,False,desktop,Mobilier d'intérieur,False,NaN,Chrome 96.0.4664,NaN,Linge de maison,Spanish,ES,WilTec (ES),False,1,FALSE,"13,99","50,17",mastercard,Windows 10,SHOPBOT,False,11 - 20,1 - 100,False,Online Experience > Ease of Use > 1;Support > ...,NaN
32974,369045220,Always correct and punctual!,Sempre corretti e puntuali!,10,NPS,Customer Effort Score,2021-11-30T22:33:37.000Z,2021-11-30,mardi,novembre,False,desktop,Jardin piscine,True,NaN,Safari 14.1.1,NaN,Plante et soin des végétaux,Italian,IT,Eacommerce (IT),False,1,FALSE,"5,9","30,86",paypal,Mac OS X 10.15.6,SHOPBOT,False,1 - 5,1 - 100,False,Delivery > Speed of delivery > 1;Delivery > Or...,NaN
32981,369057301,My payment method didn't work as planned.,Il mio metodo di pagamento non ha funzionato c...,3,NPS,Customer Effort Score,2021-11-30T22:46:29.000Z,2021-11-30,mardi,novembre,False,desktop,Luminaire,True,Ho avuto problemi ad iscrivermi o ad accedere.,Chrome 96.0.4664,NaN,Eclairage d'intér

In [32]:
df_dropped_na.describe()

,id,score,nb_articles
count,1.150700e+04,11507.000000,11507.000000
mean,3.299768e+08,8.879030,1.319458
std,2.227446e+07,1.910604,1.022843
min,2.955617e+08,0.000000,1.000000
25%,3.101173e+08,8.000000,1.000000
50%,3.283523e+08,10.000000,1.000000
75%,3.493736e+08,10.000000,1.000000
max,3.690581e+08,10.000000,27.000000


In [33]:
df_total.describe()

,id,score,nb_articles
count,9.518200e+04,95182.000000,95182.000000
mean,3.355182e+08,8.969721,1.322183
std,2.117025e+07,1.545100,1.027572
min,2.955617e+08,0.000000,1.000000
25%,3.201404e+08,8.000000,1.000000
50%,3.370718e+08,10.000000,1.000000
75%,3.552651e+08,10.000000,1.000000
max,3.690581e+08,10.000000,39.000000
